# About
this is for the week 3 project

In [1]:
import pandas as pd
import numpy as np

In [2]:
def strip(text):
    try:
        return text.strip()
    except AttributeError:
        return text

# instead of fetching from web page, cop and past it as csv
df = pd.read_csv('./toronto.csv', 
                 sep=r',', 
                 converters = {'Postcode' : strip,
                                'Borough' : strip,
                                'Neighborhood' : strip}
                 )

df = df[df['Borough'] != 'Not assigned' ]

df[df['Neighbourhood'] == 'Not assigned' ] # there is no neighbourhood no assigned

,Postcode,Borough,Neighbourhood


- there is no assigned neighbourhoods

In [3]:
df2 = pd.DataFrame({'Postcode':df.Postcode.unique()})
df2['Borough']=pd.DataFrame(list(set(df['Borough'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=pd.Series(list(set(df['Neighbourhood'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=df2['Neighborhood'].apply(lambda x: ', '.join(strip(x)))

In [23]:
df2.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights , Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [4]:
df2.shape

(103, 3)

- end of part 1

## part 2

In [14]:
# !conda install -c conda-forge geocoder --yes

In [15]:
# import geocoder
# g=geocoder.google('{}, Toronto, Ontario'.format('M3A'))
# g

In [ ]:
# latslngs = []

# postcodes = df2['Postcode']
# for postal_code in postcodes:
#     g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#     lat_lng_coords = g.latlng
#     latslngs.append(lat_lng_coords)

# latslngs

comments above are used for trying to use geocoder

prepare data frame to set the latitudes and longitudes

In [19]:
df_geo = pd.read_csv('./Geospatial_Coordinates.csv')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [44]:
lat = []
lng = []

for c in df2['Postcode']:
    row = df_geo.loc[df_geo['Postal Code'] == c]
    lat.append(row.iat[0,1])
    lng.append(row.iat[0,2])
len(lat) == len(lng) == len(df2['Postcode'])


True

In [64]:

df2['Latitude' ] = lat
df2['Longitude'] = lng

In [65]:
df2.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights , Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


- end of part 2

## part 3 map plot

cluster neighbourhood

In [66]:
from sklearn.cluster import KMeans
kclusters = 5

toronto = df2.drop(['Neighborhood', 'Postcode', 'Borough'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 4, 2, 3, 2, 1, 0, 4, 4, 2], dtype=int32)

add clustering labels

In [69]:
# add clustering labels
neighborhoods_venues_sorted = df2[['Neighborhood']]

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df2

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels
0,M3A,North York,Parkwoods,43.753259,-79.329656,4
1,M4A,North York,Victoria Village,43.725882,-79.315572,4
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,2
3,M6A,North York,"Lawrence Heights , Lawrence Manor",43.718518,-79.464763,3
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,2


plot map, to display in a visual way

In [72]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium 
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters